# TM10007 Assignment template

In [5]:
 #Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [2]:
# Data loading functions. Uncomment the one you want to use
#from adni.load_data import load_data
from brats.load_data import load_data
#from hn.load_data import load_data
#from ecg.load_data import load_data
import numpy as np

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

print(data)


test
The number of samples: 167
The number of columns: 725
              VOLUME_ET  VOLUME_NET  VOLUME_ED  VOLUME_TC  VOLUME_WT  \
ID                                                                     
TCGA-02-0006       1662         384      36268       2046      38314   
TCGA-02-0009       4362        4349      15723       8711      24434   
TCGA-02-0011      33404       48612      45798      82016     127814   
TCGA-02-0027      12114        7587      34086      19701      53787   
TCGA-02-0033      34538        7137      65653      41675     107328   
...                 ...         ...        ...        ...        ...   
TCGA-HT-8018       2093        8685      39142      10778      49920   
TCGA-HT-8111       1929         437      54079       2366      56445   
TCGA-HT-8114       8755      168606      11325     177361     188686   
TCGA-HT-8563      11757        1012     138755      12769     151524   
TCGA-HT-A61A          0       60154      76349      60154     136503   

    

In [12]:
keys = np.array(data.keys())

#data2 = data.pop('label')
#X = np.array(data2[0:1])

y = np.array(data['label'])

# X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

#print(y)
#print(X)
print(keys)

KeyError: 'label'

In [ ]:
# test 2

In [ ]:
# test 3

In [ ]:
# test 4